# 2023 Southern Methodist University Hackathon

As part of the hackathon, we were given data from the 2023 world happiness report summary and were tasked with creating a compelling analysis within 24 hours. My group decided to combine this data with two additional datasets:
* 2023 Economist Development Index - from Wikipedia
* Age of Current government (as of 2023) - from Wikipedia

## Import Data and Libraries

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import math
import seaborn as sns
import numpy as np
from shapely.geometry import Polygon

In [ ]:
# Provided happiness summary, combined with age of government
happ_2023_gov = pd.read_csv('WHR_2023_Age - Summary data for WHR_2023.csv')
happ_2023_gov.head(3)

In [ ]:
# Democracy index
demo_ind = pd.read_csv('Democracy_Index.csv', encoding = 'unicode_escape')
demo_ind['Country'] = demo_ind['Country'].str.replace('\xa0', '')
demo_ind.head(3)

In [ ]:
# Geopandas world shape file
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.head(3)

## Merge data

In [ ]:
happ_2023_gov_demo = pd.merge(happ_2023_gov, demo_ind, left_on='Code', right_on='Country Code', how='inner').drop(columns='Country Code')
happ_2023_gov_demo.head(2)

In [ ]:
happ_2023_gov_demo_geo = pd.merge(world, happ_2023_gov_demo, left_on='iso_a3', right_on='Code', how='inner')
happ_2023_gov_demo_geo.head(1)

In [ ]:
# Find difference b/w happiness and democracy index
happ_2023_gov_demo_geo['2022_happ_demo_diff'] = abs(happ_2023_gov_demo_geo['Ladder score'] - happ_2023_gov_demo_geo['2022'])
#happ_2023_gov_demo_geo['2022_happ_demo_diff_div_cor'] = happ_2023_gov_demo_geo['2022_happ_demo_diff'] / happ_2023_gov_demo_geo['Perceptions of corruption']
#happ_2023_gov_demo_geo['2022_happ_demo_diff_mul_cor'] = happ_2023_gov_demo_geo['2022_happ_demo_diff'] * happ_2023_gov_demo_geo['Perceptions of corruption']
#happ_2023_gov_demo_geo['demo_mul_corr'] = happ_2023_gov_demo_geo['2022'] * happ_2023_gov_demo_geo['Perceptions of corruption']
#happ_2023_gov_demo_geo['demo_div_corr'] = happ_2023_gov_demo_geo['2022'] / happ_2023_gov_demo_geo['Perceptions of corruption']

In [ ]:
q1_age = happ_2023_gov_demo[happ_2023_gov_demo['Age of Government'] < 29]
q2_age = happ_2023_gov_demo[(happ_2023_gov_demo['Age of Government'] >= 29) & (happ_2023_gov_demo['Age of Government'] < 36)]
q3_age = happ_2023_gov_demo[(happ_2023_gov_demo['Age of Government'] >= 36) & (happ_2023_gov_demo['Age of Government'] < 63)]
q4_age = happ_2023_gov_demo[(happ_2023_gov_demo['Age of Government'] >= 63)]

## Visualizations

In [ ]:
import matplotlib
from matplotlib import rc
matplotlib.rcParams['axes.facecolor'] = 'white'
matplotlib.rcParams['axes.grid'] = True
matplotlib.rcParams['savefig.transparent'] = True

### Scatterplots - democracy, corruption, happiness

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), layout='constrained', sharey=True);
sns.set_style('whitegrid')
sns.set_palette("Greens")
sns.scatterplot(data=happ_2023_gov_demo, x='Perceptions of corruption', y='Ladder score', ax=ax1, color='seagreen',s=40).set(title='Impact of Corruption on Happiness', ylabel='Happiness Score', xlabel='Perception of Corruption');
plt.savefig('scatter_combined.png',dpi=1200)

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), layout='constrained', sharey=True);
sns.set_palette("Greens")
sns.scatterplot(data=happ_2023_gov_demo, x='Perceptions of corruption', y='2022', ax=ax1, color='seagreen',s=40).set(title='Impact of Corruption on Democracy', ylabel='Democracy', xlabel='Perception of Corruption');
#sns.scatterplot(data=happ_2023_gov_demo, x='2022', y='Ladder score', ax=ax2, color='seagreen').set(title='Democracy', xlabel='Democracy Index');
plt.savefig('scatter_combined_demo.png',dpi=1200)

In [ ]:
fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 4), layout='constrained');
sns.set_palette("Greens")
sns.scatterplot(data=happ_2023_gov_demo, x='Perceptions of corruption', y='2022', ax=ax1, color='seagreen',s=40).set(title='Impact of Corruption on Democracy', ylabel='Democracy', xlabel='Perception of Corruption');
sns.scatterplot(data=happ_2023_gov_demo, x='Perceptions of corruption', y='Ladder score', ax=ax2, color='seagreen',s=40).set(title='Impact of Corruption on Happiness', ylabel='Happiness', xlabel='Perception of Corruption');
plt.savefig('scatter_combined_demo_comb.png',dpi=1200)

### Regime Type

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), layout='constrained');
sns.set_palette("Greens")
sns.violinplot(data=happ_2023_gov_demo, x='Regime Type', y='Ladder score', ax=ax,
              order=['Authoritarian', 'Hybrid regime', 'Flawed democracy', 'Full democracy']).set(title='Happiness by Regime Type',
                                                                                                 ylabel='Happiness Score');
plt.savefig('violin_plot.png',dpi=1200)

### Democracy/Corruption Alignment

In [ ]:
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(6, 4), layout='constrained', sharey=True);

sns.set_palette("Greens")
plt.axvline(x=0.5, c='black')
plt.axhline(y=5, c='black')
sns.scatterplot(data=happ_2023_gov_demo, x='Perceptions of corruption', y='2022', ax=ax1, 
                color='seagreen',s=40, hue='Ladder score', palette='mako').set(title='Impact of Corruption/Democracy Alignment on Happiness', ylabel='Democracy', 
                                           xlabel='Perception of Corruption');
ax1.set_xlim(0,1)
ax1.set_ylim(0,10)
ax1.legend(loc='center left', bbox_to_anchor=(1., 0.5), ncol=1, title='Happiness')

plt.savefig('alignment.png',dpi=1200)

### Mapping

In [ ]:
# Find difference between happiness score and democracy score
happ_2023_gov_demo_geo['hap_dem_diff'] = happ_2023_gov_demo_geo['Ladder score'] - happ_2023_gov_demo_geo['2022']

In [ ]:
happ_2023_gov_demo_geo['hap_dem_diff_abs']=abs(happ_2023_gov_demo_geo['hap_dem_diff'])
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5), layout='constrained')
happ_2023_gov_demo_geo.plot('hap_dem_diff_abs', legend=True, ax=ax, cmap='mako');
ax.set_axis_off();
ax.set_title('Difference Between Happiness and Democracy', fontsize=20, loc='left');
plt.savefig('happ_dem_diff.png', dpi=1200)

In [ ]:
al_type = ['High Democracy, High Corruption', 'High Democracy, Low Corruption', 'Low Democracy, High Corruption']
alignment = []
col = []
dem = happ_2023_gov_demo_geo['2022']
cor = happ_2023_gov_demo_geo['Perceptions of corruption']
for i in range(len(happ_2023_gov_demo_geo)):
    dem_i = dem[i]
    cor_i = cor[i]
    if dem_i > 5:
        if cor_i > 0.5:
            alignment.append('High Democracy, High Corruption')
            col.append('#3a305c')
        else:
            alignment.append('High Democracy, Low Corruption')
            col.append('#51779e')
    else:
        if cor_i > 0.5:
            alignment.append('Low Democracy, High Corruption')
            col.append('#7dc3ae')
        else:
            alignment.append('Low Democracy, Low Corruption')
            col.append('#7dc3ae')

In [ ]:
happ_2023_gov_demo_geo['alignment'] = alignment
happ_2023_gov_demo_geo['col'] = col

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 5), layout='constrained')
happ_2023_gov_demo_geo.plot( color=happ_2023_gov_demo_geo['col'],ax=ax, legend=True)
fig.suptitle('Democracy and Corruption Alignment')
ax.axis('off');
plt.savefig('al_map.png', dpi=1200)

### 3D Scatterplot - Happiness & Government

In [ ]:
fig = plt.figure(figsize=(8,8), layout='constrained')
ax=plt.axes(projection='3d')
scatter=ax.scatter3D(happ_2023_gov_demo_geo['2022'],happ_2023_gov_demo_geo['Age of Government'],happ_2023_gov_demo_geo['Ladder score'],
            c=happ_2023_gov_demo_geo['Perceptions of corruption'],s=40)
handles, labels = scatter.legend_elements(prop="colors", alpha=0.6)
legend = ax.legend(handles, labels, loc="center right", title="Corruption",bbox_to_anchor=(1.35, 0.5))
ax.set_xlabel('Democracy', fontsize=14)
ax.set_ylabel('Age of Government', fontsize=14)
ax.set_zlabel('Happiness', fontsize=14)
ax.set_title('Happiness & Government Stability', fontsize=18)
plt.savefig('three_d.png', dpi=1200)